In [1]:
import math
import numpy as np
from collections import namedtuple
from decimal import *

In [2]:
Direction = namedtuple('Direction', ('SW', 'NW', 'NE', 'SE'))
Direction.__add__ = lambda a, b : Direction(a.SW + b.SW, a.NW + b.NW, a.NE + b.NE, a.SE + b.SE )
Direction.__sub__ = lambda a, b : Direction(a.SW - b.SW, a.NW - b.NW, a.NE - b.NE, a.SE - b.SE )

In [3]:
DirectionPlus = namedtuple('DirectionPlus', ('SW', 'NW', 'NE', 'SE', 'IS'))
DirectionPlus.__add__ = lambda a, b : DirectionPlus(a.SW + b.SW, a.NW + b.NW, a.NE + b.NE, a.SE + b.SE, a.IS + b.IS )
DirectionPlus.__sub__ = lambda a, b : DirectionPlus(a.SW - b.SW, a.NW - b.NW, a.NE - b.NE, a.SE - b.SE, a.IS - b.IS )

# 簡易評価式の算出根拠

## 1. 算出の前提

### 1) まとめ

部位の面積及び長さ等を計算するにあたっての前提条件を以下に示す。

| 意味 | 記号| 床断熱<br>住宅 | 基礎断熱<br>住宅 | 単位 | 備考 |
| --- | :---: | :---: | :---: | :---: | --- |
| 階数 | - | 2 | 2 | - | - |
| 床面積の合計 | $A_{F,total}$ | 90.0 | 90.0 | m<sup>2</sup> | - |
| 形状係数(1階) | $SF_1$ | 1.08 | 1.08 | - | - |
| 形状係数(2階) | $SF_2$ | 1.04 | 1.04 | - | - |
| 主開口方位 | - | 南西 | 南西 | - | - |
| 1階の床面積に対する2階の床面積の比 | $r_{FA}$ | 0.77 | 0.77 | - | 自立循環型住宅プランの比を採用した。 |
| 1階の床面積(玄関・浴室除く)に対する<br>土間床面積(玄関・浴室除く)の比 | $r_{EFA}$ | 0.0 | 1.0 | - | - |
| 基礎の高さ(基礎断熱住宅) | $h_{base}$ | 0.0 | 0.5 | m | 基礎の高さとは土間基礎上端から1階床レベルまでの間を言う。<br>(GLからではない。) |
| 基礎の高さ(玄関) | $h_{base,d}$ | 0.18 | 0.0 | m | - |
| 基礎の高さ(浴室) | $h_{base,b}$ | 0.5 | 0.0 | m | - |
| 高さ(1階) | $h_1$ | 2.9 | 2.9 | m | - |
| 高さ(2階) | $h_2$ | 2.7 | 2.7 | m | - |
| ドア(玄関) | $A_{door,1}$ | 1.89 | 1.89 | m<sup>2</sup> | 方位：北西 自立循環型住宅プランの面積を採用 |
| ドア(勝手口) | $A_{door,2}$ | 1.62 | 1.62 | m<sup>2</sup> | 方位：北東 自立循環型住宅プランの面積を採用 |
| 玄関の土間床等の周辺部の基礎長さ(主開口方位側) | $L_{base,d,MS}$ | 1.365 | 0.0 | m | - |
| 玄関の土間床等の周辺部の基礎長さ(その他の方位側) | $L_{base,d,OS}$ | 1.82 | 0.0 | m | - |
| 浴室の土間床等の周辺部の基礎長さ(主開口方位側) | $L_{base,b,MS}$ | 1.82 | 0.0 | m | - |
| 浴室の土間床等の周辺部の基礎長さ(その他の方位側) | $L_{base,b,OS}$ | 1.82 | 0.0 | m | - |
| 開口部の面積比率 | $r_{opn}$ | 0.14 | 0.14 | - | 床断熱住宅と基礎断熱住宅とを同じ開口部面積にするために、<br>基礎面積を含めない外皮表面積に対する値とする。 |
| 方位別窓面積比率(南西) | $r_{wnd,SW}$ | 0.6862 | 0.6862 | - | - |
| 方位別窓面積比率(北西) | $r_{wnd,NW}$ | 0.0721 | 0.0721 | - | - |
| 方位別窓面積比率(北東) | $r_{wnd,NE}$ | 0.1097 | 0.1097 | - | - |
| 方位別窓面積比率(南東) | $r_{wnd,SE}$ | 0.1097 | 0.1097 | - | - |


※基礎の高さについて  
下記を想定すると、根太がけの場合は524～539、根太レスの場合は479～494  
・基礎の高さは400mm（参考：長期優良住宅）  
・土台は105mm又は10mm  
・根太は45mm（根太がけの場合のみ）  
・床荒板およびフローリングは24mm（12mm＋12mm）  
・GLから土間基礎上端までは50mm  
あるいは下記を想定すると、根太がけの場合は489～504、根太レスの場合は444～459  
・床下空間高さ（土間基礎上端から大引き下端まで）は330mmを確保（長期優良）  
・大引は90又は105mm  
・根太は45mm（根太がけの場合のみ）  
・床荒板およびフローリングは24mm（12mm＋12mm）  
以上を勘案して、500mmと決めた。

### 2) 辞書型変数への代入

床断熱住宅については変数( FIH( = floor_ins_house ) )に、基礎断熱住宅については変数( BIH( = base_ins_house ) )に設定する。

In [4]:
FIH = {}
BIH = {}

In [5]:
FIH['type'] = 'floor_ins'
BIH['type'] = 'base_ins'

#### a) 階数

階数は2階とする。

####  b) 床面積の合計

$ A_{F,total} = 90.0 $ (m<sup>2</sup>)

In [6]:
FIH['床面積の合計'] = Decimal('90.0')
BIH['床面積の合計'] = Decimal('90.0')

#### c) 形状係数

1階: $ SF_1 = 1.08 $  
2階: $ SF_2 = 1.04 $

In [7]:
FIH['形状係数'] = [ Decimal('1.08'), Decimal('1.04') ]
BIH['形状係数'] = [ Decimal('1.08'), Decimal('1.04') ]

#### d) 主開口方位

南西とする。

#### e) 1階の床面積に対する2階の床面積の比

自立循環型住宅プランの比を採用する。

$ r_{FA} = 0.77 $

In [8]:
FIH['1階の床面積に対する2階の床面積の比'] = Decimal('0.77')
BIH['1階の床面積に対する2階の床面積の比'] = Decimal('0.77')

#### f) 玄関と浴室を除く1階の床面積に対する土間床面積の比

$ r_{EFA} = \left \{
\begin{array}{ll}
    0.0 & ( \text{floor insulated house} ) \\
    1.0 & ( \text{base insulted house} )
\end{array}
\right.
$

In [9]:
FIH['玄関と浴室を除く1階の床面積に対する土間床面積の比'] = Decimal('0.0')
BIH['玄関と浴室を除く1階の床面積に対する土間床面積の比'] = Decimal('1.0')

#### g) 基礎の高さ

$ h_{base} = 0.5 $ (m) (一般部分)  

In [10]:
FIH['基礎の高さ'] = Decimal('0.5')
BIH['基礎の高さ'] = Decimal('0.5')

$ h_{base,entrance} = 0.18 $ (m) (玄関)

In [11]:
FIH['玄関の基礎の高さ'] = Decimal('0.18')
BIH['玄関の基礎の高さ'] = Decimal('0.18')

$ h_{base,bath} = 0.5 $ (m) (浴室)

In [12]:
FIH['浴室の基礎の高さ'] = Decimal('0.5')
BIH['浴室の基礎の高さ'] = Decimal('0.5')

#### h) 高さ

$ h_1 = 2.9 $ (1階)  
$ h_2 = 2.7 $ (2階)

In [13]:
FIH['高さ'] = [ Decimal('2.9'), Decimal('2.7') ]
BIH['高さ'] = [ Decimal('2.9'), Decimal('2.7') ]

#### i) ドアの面積

玄関の方位は北西、勝手口の方位は北東とする。自立循環型住宅プランの面積を採用した。

$ A_{door,entrance} = 1.89 $ (m<sup>2</sup>) (玄関)  

In [14]:
FIH['玄関ドアの面積'] = Decimal('1.89')
BIH['玄関ドアの面積'] = Decimal('1.89')

$ A_{door,backentrance} = 1.62 $ (m<sup>2</sup>) (勝手口)

In [15]:
FIH['勝手口ドアの面積'] = Decimal('1.62')
BIH['勝手口ドアの面積'] = Decimal('1.62')

#### j) 玄関の土間床等の周辺部の基礎長さ

$L_{base,entrance,MS}=1.365$(m)(主開口方位側)

In [16]:
FIH['主開口方位側の玄関の土間床等の周辺部の基礎長さ'] = Decimal('1.365')
BIH['主開口方位側の玄関の土間床等の周辺部の基礎長さ'] = Decimal('1.365')

$L_{base,entrance,OS}=1.82$(m)(その他方位側)

In [17]:
FIH['その他方位側の玄関の土間床等の周辺部の基礎長さ'] = Decimal('1.82')
BIH['その他方位側の玄関の土間床等の周辺部の基礎長さ'] = Decimal('1.82')

#### k) 浴室の土間床等の周辺部の基礎長さ

$L_{base,bath,MS}=1.82$(m)(主開口方位側)

In [18]:
FIH['主開口方位側の浴室の土間床等の周辺部の基礎長さ'] = Decimal('1.82')
BIH['主開口方位側の浴室の土間床等の周辺部の基礎長さ'] = Decimal('1.82')

$L_{base,bath,OS}=1.82$(m)(その他方位側)

In [19]:
FIH['その他方位側の浴室の土間床等の周辺部の基礎長さ'] = Decimal('1.82')
BIH['その他方位側の浴室の土間床等の周辺部の基礎長さ'] = Decimal('1.82')

#### l) 開口部の面積比率

$r_{opn} = 0.14$

床断熱住宅と基礎断熱住宅とを同じ開口部面積にするために、基礎面積を含めない外皮表面積に対する値とする。

In [20]:
FIH['開口部の面積比率'] = Decimal('0.14')
BIH['開口部の面積比率'] = Decimal('0.14')

#### m) 方位別窓面積比率

南西  
$r_{wnd,SW}=0.6862$  
北西  
$r_{wnd,NW}=0.0721$  
北東  
$r_{wnd,NE}=0.1097$  
南東  
$r_{wnd,SE}=0.1320$

In [21]:
FIH['方位別窓面積比率'] = Direction( Decimal('0.6862'), Decimal('0.0721'), Decimal('0.1097'), Decimal('0.1320') ) 
BIH['方位別窓面積比率'] = Direction( Decimal('0.6862'), Decimal('0.0721'), Decimal('0.1097'), Decimal('0.1320') )

#### n) まとめ

In [22]:
FIH

{'1階の床面積に対する2階の床面積の比': Decimal('0.77'),
 'type': 'floor_ins',
 'その他方位側の浴室の土間床等の周辺部の基礎長さ': Decimal('1.82'),
 'その他方位側の玄関の土間床等の周辺部の基礎長さ': Decimal('1.82'),
 '主開口方位側の浴室の土間床等の周辺部の基礎長さ': Decimal('1.82'),
 '主開口方位側の玄関の土間床等の周辺部の基礎長さ': Decimal('1.365'),
 '勝手口ドアの面積': Decimal('1.62'),
 '基礎の高さ': Decimal('0.5'),
 '床面積の合計': Decimal('90.0'),
 '形状係数': [Decimal('1.08'), Decimal('1.04')],
 '方位別窓面積比率': Direction(SW=Decimal('0.6862'), NW=Decimal('0.0721'), NE=Decimal('0.1097'), SE=Decimal('0.1320')),
 '浴室の基礎の高さ': Decimal('0.5'),
 '玄関と浴室を除く1階の床面積に対する土間床面積の比': Decimal('0.0'),
 '玄関の基礎の高さ': Decimal('0.18'),
 '玄関ドアの面積': Decimal('1.89'),
 '開口部の面積比率': Decimal('0.14'),
 '高さ': [Decimal('2.9'), Decimal('2.7')]}

In [23]:
BIH

{'1階の床面積に対する2階の床面積の比': Decimal('0.77'),
 'type': 'base_ins',
 'その他方位側の浴室の土間床等の周辺部の基礎長さ': Decimal('1.82'),
 'その他方位側の玄関の土間床等の周辺部の基礎長さ': Decimal('1.82'),
 '主開口方位側の浴室の土間床等の周辺部の基礎長さ': Decimal('1.82'),
 '主開口方位側の玄関の土間床等の周辺部の基礎長さ': Decimal('1.365'),
 '勝手口ドアの面積': Decimal('1.62'),
 '基礎の高さ': Decimal('0.5'),
 '床面積の合計': Decimal('90.0'),
 '形状係数': [Decimal('1.08'), Decimal('1.04')],
 '方位別窓面積比率': Direction(SW=Decimal('0.6862'), NW=Decimal('0.0721'), NE=Decimal('0.1097'), SE=Decimal('0.1320')),
 '浴室の基礎の高さ': Decimal('0.5'),
 '玄関と浴室を除く1階の床面積に対する土間床面積の比': Decimal('1.0'),
 '玄関の基礎の高さ': Decimal('0.18'),
 '玄関ドアの面積': Decimal('1.89'),
 '開口部の面積比率': Decimal('0.14'),
 '高さ': [Decimal('2.9'), Decimal('2.7')]}

## 2. 算出過程

途中結果プリント用関数

In [24]:
def printresult(floor_ins_result,base_ins_result):
    print('床断熱住宅: '+ floor_ins_result )
    print('基礎断熱住宅: ' + base_ins_result )

In [25]:
def printresult2(item_name):
    printresult(str(FIH[item_name]),str(BIH[item_name]))

### 1) 床面積(土間床含む)

$ \begin{align} A_{F,1} = \frac{A_{F,total}}{ ( 1 + r_{FA} )} \end{align} $

$ \begin{align} A_{F,2} = A_{F,total} - A_{F,1} \end{align} $

$A_{F,total}$: 床面積の合計(m<sup>2</sup>)  
$r_{FA}$: 1階の床面積に対する2階の床面積の比  
$A_{F,1}$: 1階の床面積(土間床含む)(m<sup>2</sup>)  
$A_{F,2}$: 2階の床面積(m<sup>2</sup>)

In [26]:
def get_A_F(A_F_total, r_FA):
    A_F_1 = A_F_total / (Decimal('1.0') + r_FA)
    A_F_2 = A_F_total - A_F_1
    return [ A_F_1, A_F_2 ]

In [27]:
FIH['床面積'] = get_A_F(FIH['床面積の合計'], FIH['1階の床面積に対する2階の床面積の比'])
BIH['床面積'] = get_A_F(BIH['床面積の合計'], BIH['1階の床面積に対する2階の床面積の比'])

In [28]:
printresult2('床面積')

床断熱住宅: [Decimal('50.84745762711864406779661017'), Decimal('39.15254237288135593220338983')]
基礎断熱住宅: [Decimal('50.84745762711864406779661017'), Decimal('39.15254237288135593220338983')]


### 2) 周長

$ \begin{align} L_{prm,i} = \sqrt{A_{F,i}} \times 4 \times SF_i \end{align} $

$L_{prm,i}$: $i$階の周長(m)($i=1,2$)  
$SF_i$: $i$階の形状係数($i=1,2$)  

In [29]:
def get_L_prm(A_F, SF):
    return [ 4 * SF_i * np.sqrt(A_F_i) for A_F_i, SF_i in zip(A_F, SF) ]

In [30]:
FIH['周長'] = get_L_prm(FIH['床面積'], FIH['形状係数'])
BIH['周長'] = get_L_prm(BIH['床面積'], BIH['形状係数'])

In [31]:
printresult2('周長')

床断熱住宅: [Decimal('30.80479821749103720382286217'), Decimal('26.02994885296810553877691029')]
基礎断熱住宅: [Decimal('30.80479821749103720382286217'), Decimal('26.02994885296810553877691029')]


### 3) 長辺の長さ

$ \displaystyle L_{LS,i} = \frac{ \frac{L_{prm,i}}{2} + \sqrt{ \left( \frac{L_{prm,i}}{2} \right)^2 - 4 \times A_{F,i} } }{2} $

$L_{LS,i}$: $i$階の長辺の長さ($i=1,2$)

In [32]:
def get_L_LS(L_prm, A_F):
    return [ ( L_prm_i / 2 + np.sqrt( ( L_prm_i / 2 )**2 - 4 * A_F_i ) ) / 2 for L_prm_i, A_F_i in zip(L_prm, A_F) ]

In [33]:
FIH['長辺の長さ'] = get_L_LS(FIH['周長'], FIH['床面積'])
BIH['長辺の長さ'] = get_L_LS(BIH['周長'], BIH['床面積'])

In [34]:
printresult2('長辺の長さ')

床断熱住宅: [Decimal('10.60998228072970436910962105'), Decimal('8.294900835666091565027726595')]
基礎断熱住宅: [Decimal('10.60998228072970436910962105'), Decimal('8.294900835666091565027726595')]


### 4) 短辺の長さ:

$ \displaystyle L_{SS,i} = \frac{L_{prm,i}}{2} - L_{LS,i} $

$L_{ss,i}$: $i$階の短辺の長さ(m)

In [35]:
def get_L_SS(L_prm, L_LS):
    return [ L_prm_i / 2 - L_LS_i for L_prm_i, L_LS_i in zip(L_prm, L_LS) ]

In [36]:
FIH['短辺の長さ'] = get_L_SS(FIH['周長'], FIH['長辺の長さ'])
BIH['短辺の長さ'] = get_L_SS(BIH['周長'], BIH['長辺の長さ'])

In [37]:
printresult2('短辺の長さ')

床断熱住宅: [Decimal('4.79241682801581423280181003'), Decimal('4.720073590817961204360728545')]
基礎断熱住宅: [Decimal('4.79241682801581423280181003'), Decimal('4.720073590817961204360728545')]


### 5) 玄関の土間床等の面積

$ A_{EF,entrance} = L_{base,entrance,MS} \times L_{base,entrance,OS} $

$A_{EF,entrance}$: 玄関の土間床等の面積(m<sup>2</sup>)  
$L_{base,entrance,MS}$: 玄関の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,entrance,OS}$: 玄関の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

In [38]:
def get_A_EF_entrance(L_base_entrance_MS, L_base_entrance_OS):
    return L_base_entrance_MS * L_base_entrance_OS

In [39]:
FIH['玄関の土間床等の面積'] = get_A_EF_entrance(FIH['主開口方位側の玄関の土間床等の周辺部の基礎長さ'], FIH['その他方位側の玄関の土間床等の周辺部の基礎長さ'])
BIH['玄関の土間床等の面積'] = get_A_EF_entrance(BIH['主開口方位側の玄関の土間床等の周辺部の基礎長さ'], BIH['その他方位側の玄関の土間床等の周辺部の基礎長さ'])

In [40]:
printresult2('玄関の土間床等の面積')

床断熱住宅: 2.48430
基礎断熱住宅: 2.48430


### 6) 浴室の土間床等の面積

$ A_{EF,bath} = L_{base,bath,MS} \times L_{base,bath,OS} $

$A_{EF,bath}$: 浴室の土間床等の面積(m<sup>2</sup>)  
$L_{base,bath,MS}$: 浴室の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,bath,OS}$: 浴室の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

In [41]:
def get_A_EF_bath(L_base_bath_MS, L_base_bath_OS):
    return L_base_bath_MS * L_base_bath_OS

In [42]:
FIH['浴室の土間床等の面積'] = get_A_EF_bath(FIH['主開口方位側の浴室の土間床等の周辺部の基礎長さ'], FIH['その他方位側の浴室の土間床等の周辺部の基礎長さ'])
BIH['浴室の土間床等の面積'] = get_A_EF_bath(BIH['主開口方位側の浴室の土間床等の周辺部の基礎長さ'], BIH['その他方位側の浴室の土間床等の周辺部の基礎長さ'])

In [43]:
printresult2('浴室の土間床等の面積')

床断熱住宅: 3.3124
基礎断熱住宅: 3.3124


### 7) 玄関・浴室除くその他の土間床等の面積

$ A_{EF,other} = ( A_{F,1} - A_{EF,entrance} - A_{EF,bath} ) \times r_{EFA} $

$ A_{EF,other} $: 土間床等の面積(玄関・浴室除く)(m<sup>2</sup>)  
$ r_{EFA} $: 1階の床面積(玄関・浴室除く)に対する土間床面積(玄関・浴室除く)の比

In [44]:
def get_A_EF_other(A_F, A_EF_entrance, A_EF_bath, r_EFA):
    return ( A_F[0] - A_EF_entrance - A_EF_bath ) * r_EFA

In [45]:
FIH['その他の土間床等の面積'] = get_A_EF_other(FIH['床面積'], FIH['玄関の土間床等の面積'], FIH['浴室の土間床等の面積'], FIH['玄関と浴室を除く1階の床面積に対する土間床面積の比'])
BIH['その他の土間床等の面積'] = get_A_EF_other(BIH['床面積'], BIH['玄関の土間床等の面積'], BIH['浴室の土間床等の面積'], BIH['玄関と浴室を除く1階の床面積に対する土間床面積の比'])

In [46]:
printresult2('その他の土間床等の面積')

床断熱住宅: 0E-27
基礎断熱住宅: 45.05075762711864406779661017


### 8) 土間床等の面積の合計

$ A_{EF,total} = A_{EF,entrane} + A_{EF,bath} + A_{EF,other} $

$A_{EF,total}$ : 土間床等の面積の合計(m<sup>2</sup>)

In [47]:
def get_A_EF_total(A_EF_entrance, A_EF_bath, A_EF_other):
    return A_EF_entrance + A_EF_bath + A_EF_other

In [48]:
FIH['土間床等の面積の合計'] = get_A_EF_total(FIH['玄関の土間床等の面積'], FIH['浴室の土間床等の面積'], FIH['その他の土間床等の面積'])
BIH['土間床等の面積の合計'] = get_A_EF_total(BIH['玄関の土間床等の面積'], BIH['浴室の土間床等の面積'], BIH['その他の土間床等の面積'])

In [49]:
printresult2('土間床等の面積の合計')

床断熱住宅: 5.796700000000000000000000000
基礎断熱住宅: 50.84745762711864406779661017


### 9) 玄関等の土間床等の外周部の長さ

南西: $L_{base,bath,SW}=0.0$  
北西: $L_{base,bath,NW}=L_{base,bath,OS}$  
北東: $L_{base,bath,NE}=L_{base,bath,MS}$  
南東: $L_{base,bath,SE}=0.0$  

室内側: 
$
L_{base,bath,IS} = \left \{
\begin{array}{ll}
    L_{base,bath,MS} + L_{base,bath,OS} & ( \text{床断熱住宅} ) \\
    0.0 & ( \text{基礎断熱住宅} )
\end{array}
\right.
$

In [50]:
def get_L_base_bath(L_base_bath_MS, L_base_bath_OS, ins_type):
    return DirectionPlus( Decimal('0.0'),
                          L_base_bath_OS,
                          L_base_bath_MS,
                          Decimal('0.0'),
                          { 'floor_ins' : L_base_bath_MS + L_base_bath_OS, 'base_ins'  : Decimal('0.0') }[ins_type] )

In [51]:
FIH['玄関等の土間床等の外周部の長さ'] = get_L_base_bath(FIH['主開口方位側の玄関の土間床等の周辺部の基礎長さ'], FIH['その他方位側の玄関の土間床等の周辺部の基礎長さ'], FIH['type'])
BIH['玄関等の土間床等の外周部の長さ'] = get_L_base_bath(BIH['主開口方位側の玄関の土間床等の周辺部の基礎長さ'], BIH['その他方位側の玄関の土間床等の周辺部の基礎長さ'], BIH['type'] )

In [52]:
printresult2('玄関等の土間床等の外周部の長さ')

床断熱住宅: DirectionPlus(SW=Decimal('0.0'), NW=Decimal('1.82'), NE=Decimal('1.365'), SE=Decimal('0.0'), IS=Decimal('3.185'))
基礎断熱住宅: DirectionPlus(SW=Decimal('0.0'), NW=Decimal('1.82'), NE=Decimal('1.365'), SE=Decimal('0.0'), IS=Decimal('0.0'))


### 10) 浴室の土間床等の外周部の長さ

南西:  $L_{base,bath,SW}=0.0$  
北西:  $L_{base,bath,NW}=L_{base,bath,OS}$  
北東:  $L_{base,bath,NE}=L_{base,bath,MS}$  
南東:  $L_{base,bath,SE}=0.0$

室内側: 
$
L_{base,bath,IS} = \left \{
\begin{array}{ll}
    L_{base,bath,MS} + L_{base,bath,OS} & ( \text{床断熱住宅} ) \\
    0.0 & ( \text{基礎断熱住宅} )
\end{array}
\right.
$

In [53]:
def get_L_base_bath(L_base_bath_MS, L_base_bath_OS, ins_type):
    return DirectionPlus( Decimal('0.0'),
                          L_base_bath_OS,
                          L_base_bath_MS,
                          Decimal('0.0'),
                          { 'floor_ins' : L_base_bath_MS + L_base_bath_OS, 'base_ins'  : Decimal('0.0') }[ins_type] )

In [54]:
FIH['浴室の土間床等の外周部の長さ'] = get_L_base_bath(FIH['主開口方位側の浴室の土間床等の周辺部の基礎長さ'], FIH['その他方位側の浴室の土間床等の周辺部の基礎長さ'], FIH['type'])
BIH['浴室の土間床等の外周部の長さ'] = get_L_base_bath(BIH['主開口方位側の浴室の土間床等の周辺部の基礎長さ'], BIH['その他方位側の浴室の土間床等の周辺部の基礎長さ'], BIH['type'])

In [55]:
printresult2('浴室の土間床等の外周部の長さ')

床断熱住宅: DirectionPlus(SW=Decimal('0.0'), NW=Decimal('1.82'), NE=Decimal('1.82'), SE=Decimal('0.0'), IS=Decimal('3.64'))
基礎断熱住宅: DirectionPlus(SW=Decimal('0.0'), NW=Decimal('1.82'), NE=Decimal('1.82'), SE=Decimal('0.0'), IS=Decimal('0.0'))


### 11) 玄関・浴室除くその他の土間床等の外周部の長さ

南西: $L_{base,SW} = \left \{
\begin{array}{ll}
    0.0 & \text{(床断熱住宅)}\\
    L_{LS,1} - L_{base,entrance,SW} - L_{base,bath,SW} & \text{(基礎断熱住宅)}
\end{array}
\right.
$

北西: $L_{base,NW} = \left \{
\begin{array}{ll}
    0.0 & \text{(床断熱住宅)}\\
    L_{SS,1} - L_{base,entrance,NW} - L_{base,bath,NW} & \text{(基礎断熱住宅)}
\end{array}
\right.
$

北東: $L_{base,NE} = \left \{
\begin{array}{ll}
    0.0 & \text{(床断熱住宅)}\\
    L_{LS,1} - L_{base,entrance,NE} - L_{base,bath,NE} & \text{(基礎断熱住宅)}
\end{array}
\right.
$

南東: $L_{base,SE} = \left \{
\begin{array}{ll}
    0.0 & \text{(床断熱住宅)}\\
    L_{SS,1} - L_{base,entrance,SE} - L_{base,bath,SE} & \text{(基礎断熱住宅)}
\end{array}
\right.
$

室内側: $ L_{base,IS} = 0.0 $

In [56]:
def get_L_base_other(L_LS, L_SS, L_base_entrance, L_base_bath,ins_type):
    return {
        'floor_ins' : DirectionPlus( Decimal('0.0'), Decimal('0.0'), Decimal('0.0'), Decimal('0.0'), Decimal('0.0') ),
        'base_ins'  : DirectionPlus( L_LS[0] - L_base_entrance.SW - L_base_bath.SW,
                                     L_SS[0] - L_base_entrance.NW - L_base_bath.NW,
                                     L_LS[0] - L_base_entrance.NE - L_base_bath.NE,
                                     L_SS[0] - L_base_entrance.SE - L_base_bath.SE,
                                     Decimal('0.0') )
    }[ins_type]

In [57]:
FIH['その他の土間床等の外周部の長さ'] = get_L_base_other(FIH['長辺の長さ'], FIH['短辺の長さ'], FIH['玄関等の土間床等の外周部の長さ'], FIH['浴室の土間床等の外周部の長さ'], FIH['type'] )
BIH['その他の土間床等の外周部の長さ'] = get_L_base_other(BIH['長辺の長さ'], BIH['短辺の長さ'], BIH['玄関等の土間床等の外周部の長さ'], BIH['浴室の土間床等の外周部の長さ'], BIH['type'] ) 

In [58]:
printresult2('その他の土間床等の外周部の長さ')

床断熱住宅: DirectionPlus(SW=Decimal('0.0'), NW=Decimal('0.0'), NE=Decimal('0.0'), SE=Decimal('0.0'), IS=Decimal('0.0'))
基礎断熱住宅: DirectionPlus(SW=Decimal('10.60998228072970436910962105'), NW=Decimal('1.15241682801581423280181003'), NE=Decimal('7.42498228072970436910962105'), SE=Decimal('4.79241682801581423280181003'), IS=Decimal('0.0'))


### 12) 土間床周長(合計)

南西:   $ L_{prm,SW} = L_{base,SW} + L_{base,entrance,SW} + L_{base,bath,SW} $  
北西:   $ L_{prm,NW} = L_{base,NW} + L_{base,entrance,NW} + L_{base,bath,NW} $  
北東:   $ L_{prm,NE} = L_{base,NE} + L_{base,entrance,NE} + L_{base,bath,NE} $  
南東:   $ L_{prm,SE} = L_{base,SE} + L_{base,entrance,SE} + L_{base,bath,SE} $  
室内側: $ L_{prm,IS} = L_{base,IS} + L_{base,entrance,IS} + L_{base,bath,IS} $

In [59]:
def get_L_prm(L_base, L_base_d,L_base_b):
    return DirectionPlus( L_base.SW + L_base_d.SW + L_base_b.SW,
                          L_base.NW + L_base_d.NW + L_base_b.NW,
                          L_base.NE + L_base_d.NE + L_base_b.NE,
                          L_base.SE + L_base_d.SE + L_base_b.SE,
                          L_base.IS + L_base_d.IS + L_base_b.IS )

In [60]:
FIH['土間床周長'] = get_L_prm( FIH['その他の土間床等の外周部の長さ'], FIH['玄関等の土間床等の外周部の長さ'], FIH['浴室の土間床等の外周部の長さ'] )
BIH['土間床周長'] = get_L_prm( BIH['その他の土間床等の外周部の長さ'], BIH['玄関等の土間床等の外周部の長さ'], BIH['浴室の土間床等の外周部の長さ'] )

In [61]:
printresult2('土間床周長')

床断熱住宅: DirectionPlus(SW=Decimal('0.0'), NW=Decimal('3.64'), NE=Decimal('3.185'), SE=Decimal('0.0'), IS=Decimal('6.825'))
基礎断熱住宅: DirectionPlus(SW=Decimal('10.60998228072970436910962105'), NW=Decimal('4.79241682801581423280181003'), NE=Decimal('10.60998228072970436910962105'), SE=Decimal('4.79241682801581423280181003'), IS=Decimal('0.0'))


### 13) 床断熱した床の面積

$ A_{IF} = A_{F,1} - A_{EF,total} $ 

$A_{IF}$: 床断熱した床の面積(m<sup>2</sup>)

In [62]:
def get_A_IF(A_F, A_EF_total):
    return A_F[0] - A_EF_total

In [63]:
FIH['床断熱した床の面積'] = get_A_IF(FIH['床面積'], FIH['土間床等の面積の合計'])
BIH['床断熱した床の面積'] = get_A_IF(BIH['床面積'], BIH['土間床等の面積の合計'])

In [64]:
printresult2('床断熱した床の面積')

床断熱住宅: 45.05075762711864406779661017
基礎断熱住宅: 0E-26


### 14) 天井・屋根(下屋を含む)

天井・屋根(下屋を含む)の面積は、オーバーハング床を想定しない限り、1階の床面積(土間床含む)と等しい。

$ \begin{align} A_{roof} = A_{F,1} \end{align} $ 

$A_{roof}$: 天井・屋根(下屋を含む)の面積 (m<sup>2</sup>)

In [65]:
def get_A_roof(A_F):
    return A_F[0]

In [66]:
FIH['屋根又は天井の面積'] = get_A_roof(FIH['床面積'])
BIH['屋根又は天井の面積'] = get_A_roof(BIH['床面積'])

In [67]:
printresult2('屋根又は天井の面積')

床断熱住宅: 50.84745762711864406779661017
基礎断熱住宅: 50.84745762711864406779661017


### 15) 基礎(玄関)の面積

南西:  $ A_{base,entrance,SW} = L_{base,entrance,SW} \times h_{base,entrance} $  
北西:  $ A_{base,entrance,NW} = L_{base,entrance,NW} \times h_{base,entrance} $  
北東:  $ A_{base,entrance,NE} = L_{base,entrance,NE} \times h_{base,entrance} $  
南東:  $ L_{base,entrance,SE} = L_{base,entrance,SE} \times h_{base,entrance} $  
室内側:  $ L_{base,entrance,IS} = L_{base,entrance,IS} \times h_{base,entrance} $

$A_{base,entrance,SW}$: 基礎(玄関)(南西)の面積(m<sup>2</sup>)  
$A_{base,entrance,NW}$: 基礎(玄関)(北西)の面積(m<sup>2</sup>)  
$A_{base,entrance,NE}$: 基礎(玄関)(北東)の面積(m<sup>2</sup>)  
$A_{base,entrance,SE}$: 基礎(玄関)(南東)の面積(m<sup>2</sup>)  
$A_{base,entrance,IS}$: 基礎(玄関)(室内側)の面積(m<sup>2</sup>)  

In [68]:
def get_A_base_entrance(L_base_entrance, h_base_entrance):
    return DirectionPlus( L_base_entrance.SW * h_base_entrance,
                          L_base_entrance.NW * h_base_entrance,
                          L_base_entrance.NE * h_base_entrance,
                          L_base_entrance.SE * h_base_entrance,
                          L_base_entrance.IS * h_base_entrance )

In [69]:
FIH['玄関等の基礎の面積'] = get_A_base_entrance(FIH['玄関等の土間床等の外周部の長さ'], FIH['玄関の基礎の高さ'] )
BIH['玄関等の基礎の面積'] = get_A_base_entrance(BIH['玄関等の土間床等の外周部の長さ'], BIH['玄関の基礎の高さ']  )

In [70]:
printresult2('玄関等の基礎の面積')

床断熱住宅: DirectionPlus(SW=Decimal('0.000'), NW=Decimal('0.3276'), NE=Decimal('0.24570'), SE=Decimal('0.000'), IS=Decimal('0.57330'))
基礎断熱住宅: DirectionPlus(SW=Decimal('0.000'), NW=Decimal('0.3276'), NE=Decimal('0.24570'), SE=Decimal('0.000'), IS=Decimal('0.000'))


### 16) 基礎(浴室)の面積

南西:  $ A_{base,bath,SW} = L_{base,bath,SW} \times h_{base,bath} $  
北西:  $ A_{base,bath,NW} = L_{base,bath,NW} \times h_{base,bath} $  
北東:  $ A_{base,bath,NE} = L_{base,bath,NE} \times h_{base,bath} $  
南東:  $ L_{base,bath,SE} = L_{base,bath,SE} \times h_{base,bath} $  
室内側:  $ L_{base,bath,IS} = L_{base,bath,IS} \times h_{base,bath} $

$A_{base,bath,SW}$: 基礎(浴室)(南西)の面積(m<sup>2</sup>)  
$A_{base,bath,NW}$: 基礎(浴室)(北西)の面積(m<sup>2</sup>)  
$A_{base,bath,NE}$: 基礎(浴室)(北東)の面積(m<sup>2</sup>)  
$A_{base,bath,SE}$: 基礎(浴室)(南東)の面積(m<sup>2</sup>)  
$A_{base,bath,IS}$: 基礎(浴室)(室内側)の面積(m<sup>2</sup>)  

In [71]:
def get_A_base_bath(L_base_bath, h_base_bath):
    return DirectionPlus( L_base_bath.SW * h_base_bath,
                          L_base_bath.NW * h_base_bath,
                          L_base_bath.NE * h_base_bath,
                          L_base_bath.SE * h_base_bath,
                          L_base_bath.IS * h_base_bath )

In [72]:
FIH['浴室の基礎の面積'] = get_A_base_bath(FIH['浴室の土間床等の外周部の長さ'], FIH['浴室の基礎の高さ'] )
BIH['浴室の基礎の面積'] = get_A_base_bath(BIH['浴室の土間床等の外周部の長さ'], BIH['浴室の基礎の高さ'] )

In [73]:
printresult2('浴室の基礎の面積')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.910'), NE=Decimal('0.910'), SE=Decimal('0.00'), IS=Decimal('1.820'))
基礎断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.910'), NE=Decimal('0.910'), SE=Decimal('0.00'), IS=Decimal('0.00'))


### 17) 基礎断熱部分(玄関・浴室除く)の面積

南西: $ A_{base,SW} = L_{base,SW} \times h_{base} $  
北西:  $ A_{base,NW} = L_{base,NW} \times h_{base} $  
北東:  $ A_{base,NE} = L_{base,NE} \times h_{base} $  
南東:  $ A_{base,SE} = L_{base,SE} \times h_{base} $  
室内側:  $ A_{base,IS} = 0.0 $

$A_{base,SW}$: 基礎断熱部分(玄関・浴室除く)(南西)の面積(m<sup>2</sup>)  
$A_{base,NW}$: 基礎断熱部分(玄関・浴室除く)(北西)の面積(m<sup>2</sup>)  
$A_{base,NE}$: 基礎断熱部分(玄関・浴室除く)(北東)の面積(m<sup>2</sup>)  
$A_{base,SE}$: 基礎断熱部分(玄関・浴室除く)(南東)の面積(m<sup>2</sup>)  
$A_{base,IS}$: 基礎断熱部分(玄関・浴室除く)(室内側)の面積(m<sup>2</sup>)  

In [74]:
def get_A_base_other(L_base, h_base):
    return DirectionPlus( L_base.SW * h_base,
                          L_base.NW * h_base,
                          L_base.NE * h_base,
                          L_base.SE * h_base,
                          Decimal('0.0') )

In [75]:
FIH['その他の基礎の面積'] = get_A_base_other(FIH['その他の土間床等の外周部の長さ'], FIH['基礎の高さ'] )
BIH['その他の基礎の面積'] = get_A_base_other(BIH['その他の土間床等の外周部の長さ'], BIH['基礎の高さ'] )

In [76]:
printresult2('その他の基礎の面積')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.00'), NE=Decimal('0.00'), SE=Decimal('0.00'), IS=Decimal('0.0'))
基礎断熱住宅: DirectionPlus(SW=Decimal('5.304991140364852184554810525'), NW=Decimal('0.576208414007907116400905015'), NE=Decimal('3.712491140364852184554810525'), SE=Decimal('2.396208414007907116400905015'), IS=Decimal('0.0'))


### 18) 基礎断熱部分の面積の合計

南西:  $ A_{prm,SW} = A_{base,SW} + A_{base,entrance,SW} + A_{base,b,SW} $  
北西:  $ A_{prm,NW} = A_{base,NW} + A_{base,entrance,NW} + A_{base,b,NW} $  
北東:  $ A_{prm,NE} = A_{base,NE} + A_{base,entrance,NE} + A_{base,b,NE} $  
南東:  $ A_{prm,SE} = A_{base,SE} + A_{base,entrance,SE} + A_{base,b,SE} $  
室内側:  $ A_{prm,IS} = A_{base,IS} + A_{base,entrance,IS} + A_{base,b,IS} $

$A_{prm,SW}$: 基礎断熱部分(南西)の面積の合計(m<sup>2</sup>)  
$A_{prm,NW}$: 基礎断熱部分(北西)の面積の合計(m<sup>2</sup>)  
$A_{prm,NE}$: 基礎断熱部分(北東)の面積の合計(m<sup>2</sup>)  
$A_{prm,SE}$: 基礎断熱部分(南東)の面積の合計(m<sup>2</sup>)  
$A_{prm,IS}$: 基礎断熱部分(室内側)の面積の合計(m<sup>2</sup>)  

In [77]:
def get_A_base_total(A_base, A_base_entrance, A_base_bath):
    # A_base, A_base_entrance and A_base_bath are DirectionPlus namedtuple
    return A_base + A_base_entrance + A_base_bath

In [78]:
FIH['基礎の面積の合計'] = get_A_base_total(FIH['その他の基礎の面積'], FIH['玄関等の基礎の面積'], FIH['浴室の基礎の面積'] )
BIH['基礎の面積の合計'] = get_A_base_total(BIH['その他の基礎の面積'], BIH['玄関等の基礎の面積'], BIH['浴室の基礎の面積'] )

In [79]:
printresult2('基礎の面積の合計')

床断熱住宅: DirectionPlus(SW=Decimal('0.000'), NW=Decimal('1.2376'), NE=Decimal('1.15570'), SE=Decimal('0.000'), IS=Decimal('2.39330'))
基礎断熱住宅: DirectionPlus(SW=Decimal('5.304991140364852184554810525'), NW=Decimal('1.813808414007907116400905015'), NE=Decimal('4.868191140364852184554810525'), SE=Decimal('2.396208414007907116400905015'), IS=Decimal('0.000'))


### 19) 壁面積(開口部含む)

1階:  
南西: $ A_{srf,1,SW} = L_{LS,1} \times h_1 $  
北西: $ A_{srf,1,NW} = L_{SS,1} \times h_1 $  
北東: $ A_{srf,1,NE} = L_{LS,1} \times h_1 $  
北西: $ A_{srf,1,SE} = L_{SS,1} \times h_1 $

2階:  
南西: $ A_{srf,2,SW} = L_{LS,2} \times h_2 $  
北西: $ A_{srf,2,NW} = L_{SS,2} \times h_2 $  
北東: $ A_{srf,2,NE} = L_{LS,2} \times h_2 $  
北西: $ A_{srf,2,SE} = L_{SS,2} \times h_2 $

In [80]:
def get_A_srf(L_LS, L_SS, h):
    return [
        Direction( L_LS[0] * h[0], L_SS[0] * h[0], L_LS[0] * h[0], L_SS[0] * h[0] ),
        Direction( L_LS[1] * h[1], L_SS[1] * h[1], L_LS[1] * h[1], L_SS[1] * h[1] )
    ]

In [81]:
FIH['垂直外皮面積'] = get_A_srf( FIH['長辺の長さ'], FIH['短辺の長さ'], FIH['高さ'] )
BIH['垂直外皮面積'] = get_A_srf( BIH['長辺の長さ'], BIH['短辺の長さ'], BIH['高さ'] )

In [82]:
printresult2('垂直外皮面積')

床断熱住宅: [Direction(SW=Decimal('30.76894861411614267041790104'), NW=Decimal('13.89800880124586127512524909'), NE=Decimal('30.76894861411614267041790104'), SE=Decimal('13.89800880124586127512524909')), Direction(SW=Decimal('22.39623225629844722557486181'), NW=Decimal('12.74419869520849525177396707'), NE=Decimal('22.39623225629844722557486181'), SE=Decimal('12.74419869520849525177396707'))]
基礎断熱住宅: [Direction(SW=Decimal('30.76894861411614267041790104'), NW=Decimal('13.89800880124586127512524909'), NE=Decimal('30.76894861411614267041790104'), SE=Decimal('13.89800880124586127512524909')), Direction(SW=Decimal('22.39623225629844722557486181'), NW=Decimal('12.74419869520849525177396707'), NE=Decimal('22.39623225629844722557486181'), SE=Decimal('12.74419869520849525177396707'))]


### 20) 外皮面積の合計(基礎の面積を除く)

$ A'_{env} = A_{IF} + A_{EF,total} + A_{roof}
+ A_{srf,1,SW} + A_{srf,1,NW} + A_{srf,1,NE} + A_{srf,1,SE}
+ A_{srf,2,SW} + A_{srf,2,NW} + A_{srf,2,NE} + A_{srf,2,SE} $

$ A'_{env}$: 外皮面積の合計(基礎の面積を除く)(m<sup>2</sup>)

In [83]:
def get_A_dash_env(A_IF, A_EF_total, A_roof, A_srf):
    return A_IF + A_EF_total + A_roof + sum(A_srf[0]) + sum(A_srf[1])  

In [84]:
FIH['基礎を除く外皮の部位の面積の合計'] = get_A_dash_env( FIH['床断熱した床の面積'], FIH['土間床等の面積の合計'], FIH['屋根又は天井の面積'], FIH['垂直外皮面積'] )
BIH['基礎を除く外皮の部位の面積の合計'] = get_A_dash_env( BIH['床断熱した床の面積'], BIH['土間床等の面積の合計'], BIH['屋根又は天井の面積'], BIH['垂直外皮面積'] )

In [85]:
printresult2('基礎を除く外皮の部位の面積の合計')

床断熱住宅: 261.3096919879751809813771784
基礎断熱住宅: 261.3096919879751809813771784


### 21) 外皮面積の合計(m<sup>2</sup>)

$ A_{env} = A'_{env} + A_{prm,SW} + A_{prm,NW} + A_{prm,NE} + A_{prm,SE} + A_{prm,IS} $

$ A_{env} $: 外皮面積の合計(m<sup>2</sup>)

In [86]:
def get_A_env(A_dash_env, A_prm):
    return A_dash_env + sum(A_prm)

In [87]:
FIH['外皮の部位の面積の合計'] = get_A_env(FIH['基礎を除く外皮の部位の面積の合計'], FIH['基礎の面積の合計'] )
BIH['外皮の部位の面積の合計'] = get_A_env(BIH['基礎を除く外皮の部位の面積の合計'], BIH['基礎の面積の合計'] )

In [88]:
printresult2('外皮の部位の面積の合計')

床断熱住宅: 266.0962919879751809813771784
基礎断熱住宅: 275.6928910967206995832886095


### 22) 開口部の面積の合計

$ A_{opn} = A'_{env} \times r_{opn} $

$A_{opn}$: 開口部の面積の合計(m<sup>2</sup>)

In [89]:
def get_A_opn(A_dash_env, r_opn):
    return A_dash_env *  r_opn

In [90]:
FIH['開口部の面積の合計'] = get_A_opn( FIH['基礎を除く外皮の部位の面積の合計'], FIH['開口部の面積比率'] )
BIH['開口部の面積の合計'] = get_A_opn( BIH['基礎を除く外皮の部位の面積の合計'], BIH['開口部の面積比率'] )

In [91]:
printresult2('開口部の面積の合計')

床断熱住宅: 36.58335687831652533739280498
基礎断熱住宅: 36.58335687831652533739280498


### 23) 窓の面積の合計

$ A_{wnd,total} = A_{opn} - A_{door,entrance} - A_{door,backentrance} $

$A_{wnd,total}$: 窓の面積(m<sup>2</sup>)

In [92]:
def get_A_wnd_total(A_opn, A_door_entrance, A_door_backentrance):
    return A_opn - A_door_entrance - A_door_backentrance

In [93]:
FIH['窓の面積の合計'] = get_A_wnd_total( FIH['開口部の面積の合計'], FIH['玄関ドアの面積'], FIH['勝手口ドアの面積'] )
BIH['窓の面積の合計'] = get_A_wnd_total( BIH['開口部の面積の合計'], BIH['玄関ドアの面積'], BIH['勝手口ドアの面積'] )

In [94]:
printresult2('窓の面積の合計')

床断熱住宅: 33.07335687831652533739280498
基礎断熱住宅: 33.07335687831652533739280498


### 24) 窓の面積

南西側: $ A_{wnd,SW} = A_{wnd} \times r_{wnd,SW} $  
北西側: $ A_{wnd,NW} = A_{wnd} \times r_{wnd,NW} $  
北東側: $ A_{wnd,NE} = A_{wnd} \times r_{wnd,NE} $  
南東側: $ A_{wnd,SE} = A_{wnd} \times r_{wnd,SE} $  

$ A_{wnd,SW}$: 窓の面積(南西)(m<sup>2</sup>)  
$ A_{wnd,NW}$: 窓の面積(北西)(m<sup>2</sup>)  
$ A_{wnd,NE}$: 窓の面積(北東)(m<sup>2</sup>)  
$ A_{wnd,SE}$: 窓の面積(南東)(m<sup>2</sup>)  

In [95]:
def get_A_wnd(A_wnd_total, r_wnd):
    return Direction(A_wnd_total * r_wnd.SW,
                     A_wnd_total * r_wnd.NW,
                     A_wnd_total * r_wnd.NE,
                     A_wnd_total * r_wnd.SE )

In [96]:
FIH['窓の面積'] = get_A_wnd( FIH['窓の面積の合計'], FIH['方位別窓面積比率'] )
BIH['窓の面積'] = get_A_wnd( BIH['窓の面積の合計'], BIH['方位別窓面積比率'] )

In [97]:
printresult2('窓の面積')

床断熱住宅: Direction(SW=Decimal('22.69493748990079968651894278'), NW=Decimal('2.384589030926621476826021239'), NE=Decimal('3.628147249551322829511990706'), SE=Decimal('4.365683107937781344535850257'))
基礎断熱住宅: Direction(SW=Decimal('22.69493748990079968651894278'), NW=Decimal('2.384589030926621476826021239'), NE=Decimal('3.628147249551322829511990706'), SE=Decimal('4.365683107937781344535850257'))


### 25) ドアの面積

南西側: $ A_{door,SW} = 0.0 $  
北西側: $ A_{door,NW} = A_{door,entrance} $   
北東側: $ A_{door,NE} = A_{door,backentrance} $  
南東側: $ A_{door,SE} = 0.0 $  

$A_{door}$: ドアの面積(m<sup>2</sup>)

In [98]:
def get_A_door(A_door_entrance, A_door_backentrance):
    return Direction(Decimal('0.0'), A_door_entrance, A_door_backentrance, Decimal('0.0'))

In [99]:
FIH['ドアの面積'] = get_A_door( FIH['玄関ドアの面積'], FIH['勝手口ドアの面積'] )
BIH['ドアの面積'] = get_A_door( BIH['玄関ドアの面積'], BIH['勝手口ドアの面積'] )

In [100]:
printresult2('ドアの面積')

床断熱住宅: Direction(SW=Decimal('0.0'), NW=Decimal('1.89'), NE=Decimal('1.62'), SE=Decimal('0.0'))
基礎断熱住宅: Direction(SW=Decimal('0.0'), NW=Decimal('1.89'), NE=Decimal('1.62'), SE=Decimal('0.0'))


### 26) 壁の面積

南西側: $ A_{wall,SW} = A_{srf,1,SW} + A_{srf,2,SW} - A_{wnd,SW} - A_{door,SW} $  
北西側: $ A_{wall,NW} = A_{srf,1,NW} + A_{srf,2,NW} - A_{wnd,NW} - A_{door,NW} $  
北東側: $ A_{wall,NE} = A_{srf,1,NE} + A_{srf,2,NE} - A_{wnd,NE} - A_{door,NE} $  
南東側: $ A_{wall,SE} = A_{srf,1,SE} + A_{srf,2,SE} - A_{wnd,SE} - A_{door,SE} $  

$ A_{wall}$: 壁の面積(m<sup>2</sup>)

In [101]:
def get_A_wall(A_srf, A_wnd, A_door):
    # A_srf, A_wnd and A_door are Direction named tuple
    return A_srf[0] + A_srf[1] - A_wnd - A_door

In [102]:
FIH['壁の面積'] = get_A_wall(FIH['垂直外皮面積'], FIH['窓の面積'], FIH['ドアの面積'])
BIH['壁の面積'] = get_A_wall(BIH['垂直外皮面積'], BIH['窓の面積'], BIH['ドアの面積'])

In [103]:
printresult2('壁の面積')

床断熱住宅: Direction(SW=Decimal('30.47024338051379020947382007'), NW=Decimal('22.36761846552773505007319492'), NE=Decimal('47.91703362086326706648077214'), SE=Decimal('22.27652438851657518236336590'))
基礎断熱住宅: Direction(SW=Decimal('30.47024338051379020947382007'), NW=Decimal('22.36761846552773505007319492'), NE=Decimal('47.91703362086326706648077214'), SE=Decimal('22.27652438851657518236336590'))


## 3. 結果

### 四捨五入

建研HPで公表する値としては、さらに小数点2桁までにまとめた値とする。  
丸める場合に、round関数は、JIS Z 8401 付属書A の方法（0.5の場合は偶数側に丸める）が適用されるため、自前に作成したround2関数で丸めることとする。この場合は、端数が「.5」の場合は、必ず大きい側（切り上げ側）に丸められる。

In [104]:
def round2(x,d=0):
#    p=10**d
#    return (x*p*2+1)//2/p
    return x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)

In [105]:
def round2_Direction(x, d=0):
    return Direction( round2( x.SW, 2 ),
                      round2( x.NW, 2 ),
                      round2( x.NE, 2 ),
                      round2( x.SE, 2 ) )

In [106]:
def round2_DirectionPlus(x, d=0):
    return DirectionPlus( round2( x.SW, 2 ),
                      round2( x.NW, 2 ),
                      round2( x.NE, 2 ),
                      round2( x.SE, 2 ),
                      round2( x.IS, 2 ) )

### 1) 床面積の合計(m<sup>2</sup>)

In [107]:
FIH['rnd_床面積の合計'] = round2( FIH['床面積の合計'], 2 )
BIH['rnd_床面積の合計'] = round2( BIH['床面積の合計'], 2 )
printresult2('rnd_床面積の合計')

床断熱住宅: 90.00
基礎断熱住宅: 90.00


### 2) 外皮の部位の面積の合計(m<sup>2</sup>)

In [108]:
FIH['rnd_外皮の部位の面積の合計'] = round2( FIH['外皮の部位の面積の合計'], 2 )
BIH['rnd_外皮の部位の面積の合計'] = round2( BIH['外皮の部位の面積の合計'], 2 )
printresult2('rnd_外皮の部位の面積の合計')

床断熱住宅: 266.10
基礎断熱住宅: 275.69


### 3) 屋根又は天井(下屋を含む)の面積(m<sup>2</sup>)

In [109]:
FIH['rnd_屋根又は天井の面積'] = round2( FIH['屋根又は天井の面積'], 2 )
BIH['rnd_屋根又は天井の面積'] = round2( BIH['屋根又は天井の面積'], 2 )
printresult2('rnd_屋根又は天井の面積')

床断熱住宅: 50.85
基礎断熱住宅: 50.85


### 4) 壁の面積(m<sup>2</sup>) 

In [110]:
FIH['rnd_壁の面積'] = round2_Direction( FIH['壁の面積'], 2 )
BIH['rnd_壁の面積'] = round2_Direction( BIH['壁の面積'], 2 )
printresult2('rnd_壁の面積')

床断熱住宅: Direction(SW=Decimal('30.47'), NW=Decimal('22.37'), NE=Decimal('47.92'), SE=Decimal('22.28'))
基礎断熱住宅: Direction(SW=Decimal('30.47'), NW=Decimal('22.37'), NE=Decimal('47.92'), SE=Decimal('22.28'))


### 5) 窓の面積(m<sup>2</sup>) 

In [111]:
FIH['rnd_窓の面積'] = round2_Direction( FIH['窓の面積'], 2 )
BIH['rnd_窓の面積'] = round2_Direction( BIH['窓の面積'], 2 )
printresult2('rnd_窓の面積')

床断熱住宅: Direction(SW=Decimal('22.69'), NW=Decimal('2.38'), NE=Decimal('3.63'), SE=Decimal('4.37'))
基礎断熱住宅: Direction(SW=Decimal('22.69'), NW=Decimal('2.38'), NE=Decimal('3.63'), SE=Decimal('4.37'))


### 6) ドアの面積(m<sup>2</sup>)

In [112]:
FIH['rnd_ドアの面積'] = round2_Direction( FIH['ドアの面積'], 2 )
BIH['rnd_ドアの面積'] = round2_Direction( BIH['ドアの面積'], 2 )
printresult2('rnd_ドアの面積')

床断熱住宅: Direction(SW=Decimal('0.00'), NW=Decimal('1.89'), NE=Decimal('1.62'), SE=Decimal('0.00'))
基礎断熱住宅: Direction(SW=Decimal('0.00'), NW=Decimal('1.89'), NE=Decimal('1.62'), SE=Decimal('0.00'))


### 7) 床断熱した床の面積(m<sup>2</sup>)  

In [113]:
FIH['rnd_床断熱した床の面積'] = round2( FIH['床断熱した床の面積'], 2 )
BIH['rnd_床断熱した床の面積'] = round2( BIH['床断熱した床の面積'], 2 )
printresult2('rnd_床断熱した床の面積')

床断熱住宅: 45.05
基礎断熱住宅: 0.00


### 8) 土間床等の面積の合計(m<sup>2</sup>)  

参考値。

In [114]:
FIH['rnd_土間床等の面積の合計'] = round2( FIH['土間床等の面積の合計'], 2 )
BIH['rnd_土間床等の面積の合計'] = round2( BIH['土間床等の面積の合計'], 2 )
printresult2('rnd_土間床等の面積の合計')

床断熱住宅: 5.80
基礎断熱住宅: 50.85


### 9) 玄関の土間床等の面積(m<sup>2</sup>)

参考値。

In [115]:
FIH['rnd_玄関の土間床等の面積'] = round2( FIH['玄関の土間床等の面積'], 2 )
BIH['rnd_玄関の土間床等の面積'] = round2( BIH['玄関の土間床等の面積'], 2 )
printresult2('rnd_玄関の土間床等の面積')

床断熱住宅: 2.48
基礎断熱住宅: 2.48


### 10) 浴室の土間床等の面積(m<sup>2</sup>)

In [116]:
FIH['rnd_浴室の土間床等の面積'] = round2( FIH['浴室の土間床等の面積'], 2 )
BIH['rnd_浴室の土間床等の面積'] = round2( BIH['浴室の土間床等の面積'], 2 )
printresult2('rnd_浴室の土間床等の面積')

床断熱住宅: 3.31
基礎断熱住宅: 3.31


### 11) 基礎の面積の合計(m<sup>2</sup>)  

従来はこの値を表示していたが、2017.07から、玄関とそれ以外に分離させる。  
現在ではこの値は表示していない。

In [117]:
FIH['rnd_基礎の面積の合計'] = round2_DirectionPlus( FIH['基礎の面積の合計'], 2 )
BIH['rnd_基礎の面積の合計'] = round2_DirectionPlus( BIH['基礎の面積の合計'], 2 )
printresult2('rnd_基礎の面積の合計')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('1.24'), NE=Decimal('1.16'), SE=Decimal('0.00'), IS=Decimal('2.39'))
基礎断熱住宅: DirectionPlus(SW=Decimal('5.30'), NW=Decimal('1.81'), NE=Decimal('4.87'), SE=Decimal('2.40'), IS=Decimal('0.00'))


### 12) 玄関等の基礎の面積(m<sup>2</sup>)  

In [118]:
FIH['rnd_玄関等の基礎の面積'] = round2_DirectionPlus( FIH['玄関等の基礎の面積'], 2 )
BIH['rnd_玄関等の基礎の面積'] = round2_DirectionPlus( BIH['玄関等の基礎の面積'], 2 )
printresult2('rnd_玄関等の基礎の面積')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.33'), NE=Decimal('0.25'), SE=Decimal('0.00'), IS=Decimal('0.57'))
基礎断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.33'), NE=Decimal('0.25'), SE=Decimal('0.00'), IS=Decimal('0.00'))


### 13) 浴室の基礎の面積(m<sup>2</sup>)  

In [119]:
FIH['rnd_浴室の基礎の面積'] = round2_DirectionPlus( FIH['浴室の基礎の面積'], 2 )
BIH['rnd_浴室の基礎の面積'] = round2_DirectionPlus( BIH['浴室の基礎の面積'], 2 )
printresult2('rnd_浴室の基礎の面積')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.91'), NE=Decimal('0.91'), SE=Decimal('0.00'), IS=Decimal('1.82'))
基礎断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.91'), NE=Decimal('0.91'), SE=Decimal('0.00'), IS=Decimal('0.00'))


### 14) 玄関等を除く基礎の面積(m<sup>2</sup>)  

2017.07まで「基礎の面積の合計」のみを表示していたが、2017.07より、「基礎の面積(玄関)」と「基礎の面積(玄関除く)」に分けて記載することになった。  
簡易法の表記上、基礎の面積は、「玄関」と「（玄関以外の）その他」としてまとめる。  
本来であれば、「基礎の面積(玄関除く)」は、「基礎の面積(その他)」と「基礎の面積(浴室)」の合計とすべきところである。しかし、「基礎の面積(玄関)」と「基礎の面積(浴室)」、「基礎の面積(その他)」をそれぞれ小数点第二位まで四捨五入して合計した値は、「基礎の面積の合計」を小数点第二位まで四捨五入した値に一致しない。そこで、「基礎の面積の合計」を小数点第二位まで四捨五入した値から、「基礎の面積(玄関)」を小数点第二位まで四捨五入した値を減じることにより、従来評価に変更がでないように対処する。

In [120]:
FIH['rnd_玄関等を除く基礎の面積'] = FIH['rnd_基礎の面積の合計'] - FIH['rnd_玄関等の基礎の面積'] # variant type is namedtuple 'DirectionPlus'
BIH['rnd_玄関等を除く基礎の面積'] = BIH['rnd_基礎の面積の合計'] - BIH['rnd_玄関等の基礎の面積'] # variant type is namedtuple 'DirectionPlus'
printresult2('rnd_玄関等を除く基礎の面積')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.91'), NE=Decimal('0.91'), SE=Decimal('0.00'), IS=Decimal('1.82'))
基礎断熱住宅: DirectionPlus(SW=Decimal('5.30'), NW=Decimal('1.48'), NE=Decimal('4.62'), SE=Decimal('2.40'), IS=Decimal('0.00'))


### 15) その他の基礎の面積(m<sup>2</sup>)

In [121]:
FIH['rnd_その他の基礎の面積'] = FIH['rnd_基礎の面積の合計'] - FIH['rnd_玄関等の基礎の面積'] - FIH['rnd_浴室の基礎の面積']
    # variant type is namedtuple 'DirectionPlus'
BIH['rnd_その他の基礎の面積'] = BIH['rnd_基礎の面積の合計'] - BIH['rnd_玄関等の基礎の面積'] - BIH['rnd_浴室の基礎の面積']
    # variant type is namedtuple 'DirectionPlus'
printresult2('rnd_その他の基礎の面積')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.00'), NE=Decimal('0.00'), SE=Decimal('0.00'), IS=Decimal('0.00'))
基礎断熱住宅: DirectionPlus(SW=Decimal('5.30'), NW=Decimal('0.57'), NE=Decimal('3.71'), SE=Decimal('2.40'), IS=Decimal('0.00'))


### 16) 土間床周長の合計(m)

従来はこの値を表示していたが、2017.07から、玄関とそれ以外に分離させる。  
現在ではこの値は表示していない。

In [122]:
FIH['rnd_土間床周長'] = round2_DirectionPlus( FIH['土間床周長'], 2 )
BIH['rnd_土間床周長'] = round2_DirectionPlus( BIH['土間床周長'], 2 )
printresult2('rnd_土間床周長')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('3.64'), NE=Decimal('3.19'), SE=Decimal('0.00'), IS=Decimal('6.83'))
基礎断熱住宅: DirectionPlus(SW=Decimal('10.61'), NW=Decimal('4.79'), NE=Decimal('10.61'), SE=Decimal('4.79'), IS=Decimal('0.00'))


### 17) 玄関等の土間床等の外周部の長さ(m)

In [123]:
FIH['rnd_玄関等の土間床等の外周部の長さ'] = round2_DirectionPlus( FIH['玄関等の土間床等の外周部の長さ'], 2 )
BIH['rnd_玄関等の土間床等の外周部の長さ'] = round2_DirectionPlus( BIH['玄関等の土間床等の外周部の長さ'], 2 )
printresult2('rnd_玄関等の土間床等の外周部の長さ')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('1.82'), NE=Decimal('1.37'), SE=Decimal('0.00'), IS=Decimal('3.19'))
基礎断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('1.82'), NE=Decimal('1.37'), SE=Decimal('0.00'), IS=Decimal('0.00'))


### 18) 浴室の土間床等の外周部の長さ

In [124]:
FIH['rnd_浴室の土間床等の外周部の長さ'] = round2_DirectionPlus( FIH['浴室の土間床等の外周部の長さ'], 2 )
BIH['rnd_浴室の土間床等の外周部の長さ'] = round2_DirectionPlus( BIH['浴室の土間床等の外周部の長さ'], 2 )
printresult2('rnd_浴室の土間床等の外周部の長さ')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('1.82'), NE=Decimal('1.82'), SE=Decimal('0.00'), IS=Decimal('3.64'))
基礎断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('1.82'), NE=Decimal('1.82'), SE=Decimal('0.00'), IS=Decimal('0.00'))


### 19) 玄関等を除く土間床等の外周部の長さ(m)

2017.07まで「土間床周長の合計」のみを表示していたが、2017.07より、「土間床周長(玄関)」と「土間床周長(玄関除く)」に分けて記載することになった。  
簡易法の表記上、基礎の面積は、「玄関」と「（玄関以外の）その他」としてまとめる。  
本来であれば、「土間床周長(玄関除く)」は、「土間床周長(その他)」と「土間床周長(浴室)」の合計とすべきところである。しかし、「土間床周長(玄関)」と「土間床周長(浴室)」、「土間床周長(その他)」をそれぞれ小数点第二位まで四捨五入して合計した値は、「土間床周長の合計」を小数点第二位まで四捨五入した値に一致しない。そこで、「土間床周長の合計」を小数点第二位まで四捨五入した値から、「土間床周長(玄関)」を小数点第二位まで四捨五入した値を減じることにより、従来評価に変更がでないように対処する。

In [125]:
FIH['rnd_玄関等を除く土間床等の外周部の長さ'] = FIH['rnd_土間床周長'] - FIH['rnd_玄関等の土間床等の外周部の長さ']
    # variant type is namedtuple 'DirectionPlus'
BIH['rnd_玄関等を除く土間床等の外周部の長さ'] = BIH['rnd_土間床周長'] - BIH['rnd_玄関等の土間床等の外周部の長さ']
    # variant type is namedtuple 'DirectionPlus'
printresult2('rnd_玄関等を除く土間床等の外周部の長さ')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('1.82'), NE=Decimal('1.82'), SE=Decimal('0.00'), IS=Decimal('3.64'))
基礎断熱住宅: DirectionPlus(SW=Decimal('10.61'), NW=Decimal('2.97'), NE=Decimal('9.24'), SE=Decimal('4.79'), IS=Decimal('0.00'))


### 20) その他の土間床等の外周部の長さ(m<sup>2</sup>)

In [126]:
FIH['rnd_その他の土間床等の外周部の長さ'] = FIH['rnd_土間床周長'] - FIH['rnd_玄関等の土間床等の外周部の長さ'] - FIH['rnd_浴室の土間床等の外周部の長さ']
    # variant type is namedtuple 'DirectionPlus'
BIH['rnd_その他の土間床等の外周部の長さ'] = BIH['rnd_土間床周長'] - BIH['rnd_玄関等の土間床等の外周部の長さ'] - BIH['rnd_浴室の土間床等の外周部の長さ']
    # variant type is namedtuple 'DirectionPlus'
printresult2('rnd_その他の土間床等の外周部の長さ')

床断熱住宅: DirectionPlus(SW=Decimal('0.00'), NW=Decimal('0.00'), NE=Decimal('0.00'), SE=Decimal('0.00'), IS=Decimal('0.00'))
基礎断熱住宅: DirectionPlus(SW=Decimal('10.61'), NW=Decimal('1.15'), NE=Decimal('7.42'), SE=Decimal('4.79'), IS=Decimal('0.00'))


## 4. テスト

In [127]:
testcase = [ [ '外皮の部位の面積の合計', '266.10', '275.69', '' ],
             [ '床面積の合計',            '90.00',  '90.00', '' ],
             [ '屋根又は天井の面積',      '50.85',  '50.85', '' ],
             [ '床断熱した床の面積',      '45.05',   '0.00', '' ],
             [ '土間床等の面積の合計',     '5.80',  '50.85', '(参考)' ],
             [ '玄関の土間床等の面積',     '2.48',   '2.48', '(参考)' ],
             [ '浴室の土間床等の面積',     '3.31',   '3.31', '(2018.04から使用。)' ],
             [ '壁の面積',   Direction( '30.47', '22.37', '47.92', '22.28' ), Direction( '30.47', '22.37', '47.92', '22.28' ), '' ],
             [ 'ドアの面積', Direction(  '0.00',  '1.89',  '1.62',  '0.00' ), Direction(  '0.00',  '1.89',  '1.62',  '0.00' ), '' ],
             [ '窓の面積',   Direction( '22.69',  '2.38',  '3.63',  '4.37' ), Direction( '22.69',  '2.38',  '3.63',  '4.37' ), '' ],
             [ '基礎の面積の合計',                   DirectionPlus( '0.00', '1.24', '1.16', '0.00', '2.39' ), DirectionPlus(  '5.30', '1.81',  '4.87', '2.40', '0.00' ), '(2017.07まで使用。現在は使用していない。)' ],
             [ '玄関等の基礎の面積',                 DirectionPlus( '0.00', '0.33', '0.25', '0.00', '0.57' ), DirectionPlus(  '0.00', '0.33',  '0.25', '0.00', '0.00' ), '(2017.07より使用。)' ],
             [ '浴室の基礎の面積',                   DirectionPlus( '0.00', '0.91', '0.91', '0.00', '1.82' ), DirectionPlus(  '0.00', '0.91',  '0.91', '0.00', '0.00' ), '(参考値。)' ],
             [ '玄関等を除く基礎の面積',             DirectionPlus( '0.00', '0.91', '0.91', '0.00', '1.82' ), DirectionPlus(  '5.30', '1.48',  '4.62', '2.40', '0.00' ), '(2017.07より使用。)' ],
             [ 'その他の基礎の面積',                 DirectionPlus( '0.00', '0.00', '0.00', '0.00', '0.00' ), DirectionPlus(  '5.30', '0.57',  '3.71', '2.40', '0.00' ), '(参考値。)' ],
             [ '土間床周長',                         DirectionPlus( '0.00', '3.64', '3.19', '0.00', '6.83' ), DirectionPlus( '10.61', '4.79', '10.61', '4.79', '0.00' ), '(2017.07まで使用。現在は使用していない。)' ],
             [ '玄関等の土間床等の外周部の長さ',     DirectionPlus( '0.00', '1.82', '1.37', '0.00', '3.19' ), DirectionPlus(  '0.00', '1.82',  '1.37', '0.00', '0.00' ), '(2017.07より使用。)' ],
             [ '浴室の土間床等の外周部の長さ',       DirectionPlus( '0.00', '1.82', '1.82', '0.00', '3.64' ), DirectionPlus(  '0.00', '1.82',  '1.82', '0.00', '0.00' ), '(参考値。)' ],
             [ '玄関等を除く土間床等の外周部の長さ', DirectionPlus( '0.00', '1.82', '1.82', '0.00', '3.64' ), DirectionPlus( '10.61', '2.97',  '9.24', '4.79', '0.00' ), '(2017.07より使用。)' ],
             [ 'その他の土間床等の外周部の長さ',     DirectionPlus( '0.00', '0.00', '0.00', '0.00', '0.00' ), DirectionPlus( '10.61', '1.15',  '7.42', '4.79', '0.00' ), '(参考値。)' ],
           ]

In [128]:
def show_result(e,r):
    return 'EXPECTED = ' + str(e) + ' / RESULT = ' +  str(r) + ' / ' + ('.....OK' if abs( e - r ) < Decimal(0.000001) else '.....NG' )

for index, row in enumerate(testcase):
    print(row[0] + row[3])
    if type(row[1]) == str:
        print ( '  床断熱  : ' + show_result( Decimal(row[1]), FIH['rnd_'+row[0]] ) )
        print ( '  基礎断熱: ' + show_result( Decimal(row[2]), BIH['rnd_'+row[0]] ) )
    elif type(row[1]) == Direction:
        print ( '  床断熱  (南西): ' + show_result( Decimal(row[1].SW), FIH['rnd_'+row[0]].SW ) )
        print ( '  床断熱  (北西): ' + show_result( Decimal(row[1].NW), FIH['rnd_'+row[0]].NW ) )
        print ( '  床断熱  (北東): ' + show_result( Decimal(row[1].NE), FIH['rnd_'+row[0]].NE ) )
        print ( '  床断熱  (南東): ' + show_result( Decimal(row[1].SE), FIH['rnd_'+row[0]].SE ) )
        print ( '  基礎断熱(南西): ' + show_result( Decimal(row[2].SW), BIH['rnd_'+row[0]].SW ) )
        print ( '  基礎断熱(北西): ' + show_result( Decimal(row[2].NW), BIH['rnd_'+row[0]].NW ) )
        print ( '  基礎断熱(北東): ' + show_result( Decimal(row[2].NE), BIH['rnd_'+row[0]].NE ) )
        print ( '  基礎断熱(南東): ' + show_result( Decimal(row[2].SE), BIH['rnd_'+row[0]].SE ) )
    elif type(row[1]) == DirectionPlus:
        print ( '  床断熱  (南西): ' + show_result( Decimal(row[1].SW), FIH['rnd_'+row[0]].SW ) )
        print ( '  床断熱  (北西): ' + show_result( Decimal(row[1].NW), FIH['rnd_'+row[0]].NW ) )
        print ( '  床断熱  (北東): ' + show_result( Decimal(row[1].NE), FIH['rnd_'+row[0]].NE ) )
        print ( '  床断熱  (南東): ' + show_result( Decimal(row[1].SE), FIH['rnd_'+row[0]].SE ) )
        print ( '  床断熱  (室内): ' + show_result( Decimal(row[1].IS), FIH['rnd_'+row[0]].IS ) )
        print ( '  基礎断熱(南西): ' + show_result( Decimal(row[2].SW), BIH['rnd_'+row[0]].SW ) )
        print ( '  基礎断熱(北西): ' + show_result( Decimal(row[2].NW), BIH['rnd_'+row[0]].NW ) )
        print ( '  基礎断熱(北東): ' + show_result( Decimal(row[2].NE), BIH['rnd_'+row[0]].NE ) )
        print ( '  基礎断熱(南東): ' + show_result( Decimal(row[2].SE), BIH['rnd_'+row[0]].SE ) )
        print ( '  基礎断熱(室内): ' + show_result( Decimal(row[2].IS), BIH['rnd_'+row[0]].IS ) )
    else:
        print ( '  Format Error' )

外皮の部位の面積の合計
  床断熱  : EXPECTED = 266.10 / RESULT = 266.10 / .....OK
  基礎断熱: EXPECTED = 275.69 / RESULT = 275.69 / .....OK
床面積の合計
  床断熱  : EXPECTED = 90.00 / RESULT = 90.00 / .....OK
  基礎断熱: EXPECTED = 90.00 / RESULT = 90.00 / .....OK
屋根又は天井の面積
  床断熱  : EXPECTED = 50.85 / RESULT = 50.85 / .....OK
  基礎断熱: EXPECTED = 50.85 / RESULT = 50.85 / .....OK
床断熱した床の面積
  床断熱  : EXPECTED = 45.05 / RESULT = 45.05 / .....OK
  基礎断熱: EXPECTED = 0.00 / RESULT = 0.00 / .....OK
土間床等の面積の合計(参考)
  床断熱  : EXPECTED = 5.80 / RESULT = 5.80 / .....OK
  基礎断熱: EXPECTED = 50.85 / RESULT = 50.85 / .....OK
玄関の土間床等の面積(参考)
  床断熱  : EXPECTED = 2.48 / RESULT = 2.48 / .....OK
  基礎断熱: EXPECTED = 2.48 / RESULT = 2.48 / .....OK
浴室の土間床等の面積(2018.04から使用。)
  床断熱  : EXPECTED = 3.31 / RESULT = 3.31 / .....OK
  基礎断熱: EXPECTED = 3.31 / RESULT = 3.31 / .....OK
壁の面積
  床断熱  (南西): EXPECTED = 30.47 / RESULT = 30.47 / .....OK
  床断熱  (北西): EXPECTED = 22.37 / RESULT = 22.37 / .....OK
  床断熱  (北東): EXPECTED = 47.92 / RESULT = 47.92 / .....OK
  床